In [2]:
import pandas as pd 
import numpy as np 

with open('./stopwords-ko.txt', 'r') as s:
    stopwords=  s.readlines()

stopwords = [words.strip() for words in stopwords ]

In [3]:
import tensorflow_text as text 
import tensorflow_hub as hub 
import os 
from tensorflow.keras.callbacks import  ModelCheckpoint

In [4]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
# from konlpy.tag import Okt
from eunjeon import Mecab
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import urllib.request
from collections import Counter

import tensorflow as tf 

def pickleIO(obj, src, op="r"):
    if op=="w":
        with open(src, op + "b") as f:
            pickle.dump(obj, f)
    elif op=="r":
        with open(src, op + "b") as f:
            tmp = pickle.load(f)
        return tmp
    else:
        print("unknown operation")
        return obj
    
df= pd.DataFrame()
df_all = pickleIO(df, src='df_full.pkl',op='r')
df_train = pickleIO(df, src='df_train.pkl',op='r')
df_valid = pickleIO(df, src='df_valid.pkl',op='r')

df_train.document=  df_train.document.str.replace('[^가-힣]' , ' ')
df_valid.document=  df_valid.document.str.replace('[^가-힣]' , ' ')
df_all.document=  df_all.document.str.replace('[^가-힣]' , ' ')

df_train.document = df_train.document.str.replace('^ +', '')
df_valid.document = df_valid.document.str.replace('^ +', '')
df_all.document = df_all.document.str.replace('^ +', '')

df_train.document = df_train.document.replace('', np.nan)
df_valid.document = df_valid.document.replace('', np.nan)
df_all.document = df_all.document.replace('', np.nan)
df_train  =  df_train.dropna(how='any')
df_valid  =  df_valid.dropna(how='any')
df_all  =  df_all.dropna(how='any')
df_train = df_train.drop_duplicates()
df_valid = df_valid.drop_duplicates()
# df_all = df_all.drop_duplicates(subset='')
df_train.label = df_train.label.apply(lambda x: 1 if x=='pos' else 0 )
df_valid.label = df_valid.label.apply(lambda x: 1 if x=='pos' else 0 )
df_all.label = df_all.label.apply(lambda x: 1 if x=='pos' else 0 )
print(df_train.shape , df_valid.shape , df_all.shape)

C:\Users\USER\AppData\Local\Temp\ipykernel_8452\3645196036.py:35: FutureWarning: The default value of regex will change from True to False in a future version.
  df_train.document=  df_train.document.str.replace('[^가-힣]' , ' ')
C:\Users\USER\AppData\Local\Temp\ipykernel_8452\3645196036.py:36: FutureWarning: The default value of regex will change from True to False in a future version.
  df_valid.document=  df_valid.document.str.replace('[^가-힣]' , ' ')
C:\Users\USER\AppData\Local\Temp\ipykernel_8452\3645196036.py:37: FutureWarning: The default value of regex will change from True to False in a future version.
  df_all.document=  df_all.document.str.replace('[^가-힣]' , ' ')
C:\Users\USER\AppData\Local\Temp\ipykernel_8452\3645196036.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  df_train.document = df_train.document.str.replace('^ +', '')
C:\Users\USER\AppData\Local\Temp\ipykernel_8452\3645196036.py:40: FutureWarning: The default valu

(395536, 3) (99378, 3) (499782, 3)


In [30]:
mecab = Mecab()


df_train['tokenized'] = df_train['document'].apply(mecab.morphs)
df_train['tokenized'] = df_train['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
train_data = df_train.copy()

df_valid['tokenized'] = df_valid['document'].apply(mecab.morphs)
df_valid['tokenized'] = df_valid['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
test_data = df_valid.copy()

In [17]:
df_train.label.value_counts()
df_valid.label.value_counts()

1    49701
0    49677
Name: label, dtype: int64

In [5]:
df_train = pickleIO(df, src='df_train_fixed.pkl',op='r')
df_valid = pickleIO(df, src='df_valid_fixed.pkl',op='r')

In [6]:
df_train['fixed_token'] = df_train.tokenized.apply(lambda x: ' '.join(x))
df_valid['fixed_token'] = df_valid.tokenized.apply(lambda x: ' '.join(x))

In [7]:
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [8]:
def get_sentence_embedding(sentences):
    preprocessed_text= bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embedding(['내 이름은 권영규 ', '영규야 밥 먹어야지. 언제 까지 기다리게 할꺼야'])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8371293 , -0.25605115, -0.78577626, ..., -0.43259618,
        -0.56078607,  0.80764824],
       [-0.93711895, -0.5478241 , -0.9974927 , ..., -0.9735747 ,
        -0.75254464,  0.91107416]], dtype=float32)>

In [9]:
# bert layers 

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name= 'output')(l)

model = tf.keras.Model(inputs= [text_input], outputs=[l])

In [10]:
model.summary()


if not os.path.exists('./bert_model_acc'):
    os.mkdir('./bert_model_acc')
if not os.path.exists('./bert_model_loss'):
    os.mkdir('./bert_model_loss')
    
path_val_loss = './bert_model_loss/{epoch:02d}-{val_loss:.5f}.hdf5'

path_val_acc = './bert_model_acc/{epoch:02d}-{val_accuracy:.5f}.hdf5'

cp_callback_acc = ModelCheckpoint(
    path_val_acc, monitor = 'val_accuracy', verbose=1, save_best_only = True,
    save_weights_only=True
)
cp_callback2_loss = ModelCheckpoint(
    path_val_loss, monitor = 'val_loss', verbose=1, save_best_only = True,
    save_weights_only=True
)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'] )

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [11]:
model.fit(df_train.fixed_token, df_train.label, epochs=30, batch_size=64, validation_data=[df_valid.fixed_token, df_valid.label] , callbacks= [cp_callback_acc, cp_callback2_loss])

Epoch 1/30
   2/6181 [..............................] - ETA: 40:44 - loss: 0.9025 - accuracy: 0.5312   

KeyboardInterrupt: 